<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages and Importing libraries**

In [5]:
!pip install gekko
!pip install ipynb
import numpy as np
import time
from random import sample
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAWMILL CLASS**

In [33]:
DISTANCE = 1
DENSITY = 2
RADIO_MILL = 10

class Inventory:
  Total_Harvested = []
  Total_Residues = []
  
  #Total_Harvested = pd.DataFrame({'Unique_ID' : [],'Harvested':[]}) #initialize Total Harvested variable (for all the layers)
  #Total_Residues = pd.DataFrame({'Unique_ID' : [],'SumOfTotal':[]}) #Initialize Total Residues variable (for all the layers)
  
  def __init__(self, Av, Dist, Wt=-1, Pt=-1):
    self.AcumResidues = []  #Residues generated by harvesting
    self.Available = list()
    self.Available.append(Av.copy()) #Dataframe that contains Available wood Inventory at diferent time t
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.Candidates=[] #Candidate location for biorefinery
    self.RefCost = [] #Matrix Distance from any Mills and candidates cells to other cells
    self.Dist = Dist #Matrix Distance from any Mills and candidates cells to other cells
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Dist_Refi = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Harvested = [] #Ammount of harvested wood
    self.Harvested_Res = []  #Residues Harvested
    self.ProdType = Pt #type of product generated (pulp or saw)
    self.Radio_Refinery = 70
    self.Residues = []  #Residues generated by harvesting
    self.WoodType = Wt #type of wood needed (softwood, hardwood)
    self.Residues_Left = 0.30 #portion of residues left in the ground and not used for the biorefinery
    self.By_Mill = [] #Quantity harvested by each mills in each hexagon
    

    #initialize Harvested list similar to available 
    
    Id_li = list(self.Available[0].loc[:,("Unique_ID")])
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
    self.Harvested.append(pd.DataFrame(Data).copy())
    Data = {'Unique_ID':Id_li,'Growth':Ha_li}
    self.Growth =pd.DataFrame(Data).copy()    
    Data = {'Unique_ID':Id_li}
    self.By_Mill =pd.DataFrame(Data).copy()
    

  def Harvest_Mill(self,t, j,mode=DISTANCE):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')] #look for mill in position j
    Name_Mill =self.Mills.loc[j,('Name')] #look for mill in position j
    
    a =self.Cell_Less(Id1,t,RADIO_MILL) #look for cells between the range Radio_Mill
    self.Dist_Mill = a.loc[:, ('DEST','DIST')] #fill Distances to mills
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t+1, RADIO_MILL)
    D1 = np.sum(self.Available_Mill)
    print('**Demanda:', Dem, 'Disponible',D1, 'Year: ',t, 'Mill: ',Id1 )
    
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    self.By_Mill[Name_Mill] = Ha_li
        
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    if (mode == DISTANCE):
      prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    elif (mode ==DENSITY):
      prod0 = m.Intermediate(-m.sum([self.Available_Mill[i]*H[i] for i in range(Num)]))

    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    flat = [item for l in H for item in l]
    a = self.Dist_Mill
    a['Harvested'] = flat  
    ans = a

    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = self.Harvested[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
        self.By_Mill[Name_Mill][a2] = ans.loc[j,('Harvested')] 
      else:
        print(ans.loc[j,('DEST')], 'no hallado')
    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
        l0  = self.Available[t+1]["SumOfTotal"]<0
        
  
  def Harvest_All_Mills(self,t,k):
  #Harvest k mills at year t
    print('Stage:',t)
    self.Available.append(self.Available[t].copy())
    for i in range(k):
      print('Cosechando Sawmill:',i)
      self.Harvest_Mill(t,i,DISTANCE)

    
    self.Residues.append(self.Harvested[t].loc[:,('Unique_ID','Harvested')].copy())
    self.Harvested.append(self.Harvested[t].copy()) 
    self.Harvested[t+1].loc[:,('Harvested')] = 0
    self.Residues[t].loc[:,('Harvested')] = round(self.Residues[t].loc[:,('Harvested')]*0.05)
    
  
  def Harvest_All_Mills_All_Years(self,Y):
    N_Mills = len(self.Mills)
    for y in range(YEARS):
      self.Harvest_All_Mills(y,1)
      self.Add_Growth(y) 
    self.Residues.append(self.Residues[0].loc[:,('Unique_ID','Harvested')].copy())
    print('***y:',y)
    self.Residues[y].loc[:,('Harvested')] = 0
    


  def Growth_Update(self,Data):
    N = len(self.Growth)
    print('Numero de Filas', N)
    if self.ProdType == 'Sawmill':
      cl = 1
    elif self.ProdType == 'Pulpmill':
      cl =2
    
    Data = Data[Data['CLASS'] == cl]
    
    for i in range(N):    
      Id = self.Growth.loc[i,("Unique_ID")]
      Row = Data[Data['Unique_ID'] == Id]
      if not Row.empty:
        if self.WoodType == 'H':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,4] + Row.iloc[0,5] + 0.33 * Row.iloc[0,8]
        if self.WoodType == 'S':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,6] + Row.iloc[0,7] + 0.67 * Row.iloc[0,8]
    

  def Cell_Less(self, Id_Mill, t, Radio=  RADIO_MILL):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Total_Available(self, Id_Mill, Radio=RADIO_MILL):
    #Given a Radio and a Mill ID this function returns a the total inventory Available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    
    for j in range(len(Ans)):
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Total = np.sum(Disp)
    return(Total)


  def Cell_Less_Res(self, Id_Mill, t, Radio=50):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Forest Residues available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Disp_Radio(self, Id_Mill, t, Radio=RADIO_MILL):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)
  

  def Num_Cell(self, Id_Mill, Radio=RADIO_MILL):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    return(len(Ans))
  
  def Mills_Update(self,M):
    i1 = (M['s_h'] == self.WoodType)&(M['plant'] == self.ProdType)
    self.Mills = M.loc[i1,:]
    #self.Mills.reset_index
    
    Num = len(self.Mills)
    ind = [i for i in range(Num)]
    self.Mills.index = ind
    

  def Disp_Radio_Res(self, Id_Mill, t, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Residues Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    #update index from 0 to rows-1
    Disp =[]
    for j in range(len(Ans)):  
      #print(self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')])                                                   
      if self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)


  def Harvest_Refinery(self,t,Cand, Demanda=20000): #Evaluates Candidate location for refinery in year t
    m2 = GEKKO(remote=True)
    m2.options.solver = 3 #IPOPT solver
    m2.options.IMODE = 3 #steady state optimization

    Id1 =self.Candidates[Cand] #assign unique ID
    a =self.Cell_Less_Res(Id1,t,self.Radio_Refinery) #look for hexagons between radio distance
    self.Dist_Refi = a.loc[:, ('DEST','DIST')]  #obtain column Destination and Distance from 'a'
    Num = len(self.Dist_Refi) #calculate the number of cells betwee 
    H = m2.Array(m2.Var,(Num),value = 0)

    self.Available_Refi = self.Disp_Radio_Res(Id1, t+1, self.Radio_Refinery) #look for available quantity in cells
      
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Refi[i]  
    prod0 = m2.Intermediate(m2.sum([self.Dist_Refi.loc[i,('DIST')]*H[i] for i in range(Num)]))
    m2.Equation(Demanda ==m2.sum([H[i] for i in range(0,len(H))]))
    m2.Obj(prod0) 
    m2.solve(disp=False,debug=False)
    
    a = self.Dist_Refi
    flat = [item for l in H for item in l]
    a['Harvested'] = flat
    ans = a
        
    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested_Res[t][self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested_Res[t].loc[a2,("Harvested")] = self.Harvested_Res[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
    
    #Updating Available Inventory
    print('Residues Available Before',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")]))     
    
    for j in range(len(ans)):                                                                         
      if not(self.AcumResidues[t+1][self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.AcumResidues[t+1].loc[a2,("SumOfTotal")] = self.AcumResidues[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
    
        l0  = self.AcumResidues[t+1]["SumOfTotal"]<0
        if np.sum(self.Available[t+1].loc[:,("SumOfTotal")])<0 :
          print('****Available Negative')
    
    print('answer\n',ans)
    print('Cells Harvested',self.Harvested_Res[t][self.Harvested_Res[t]["Harvested"]>0]) 
    print('Distances',self.Dist_Refi.loc[self.Harvested_Res[t]["Harvested"]>0],('DIST')) 
    print('Total Harvested',np.sum(self.Harvested_Res[t].loc[:,("Harvested")])) 
    print('Residues Available After',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")])) 
    res = round(np.dot(flat,self.Dist_Refi.loc[:,('DIST')]))  #Delivery cost to refinery
    self.RefCost[Cand] += res
    print('Cost', self.RefCost[Cand])
  

  def Optimal_Location_Total(self,Y,NCan = 0):
    #run Ncan locations from year 0 to year 'y'
    if (NCan == 0):
      Cans = len(self.Candidates)
    else:
      Cans = NCan
    self.RefCost = [0]*Cans

    for c in range(Cans):
      self.AcumResidues = []
      self.AcumResidues.append(self.Residues[0].copy())
      self.AcumResidues[0].columns=["Unique_ID", "SumOfTotal"]
      self.AcumResidues[0].loc[:,('SumOfTotal')] = 0
        
      for i in range(Y):
        self.AcumResidues.append(self.AcumResidues[i].copy())
        self.AcumResidues[i+1].columns=["Unique_ID", "SumOfTotal"]
        self.AcumResidues[i+1]['SumOfTotal'] =self.AcumResidues[i+1]['SumOfTotal'] + self.Residues[i]['Harvested']
        self.Harvest_Refinery(i,c)
        self.Harvested_Res.append(self.Harvested_Res[i].copy())     
        self.Harvested_Res[i+1].loc[:,('Harvested')] = 0

  def Add_Growth(self,t):
    #increase the available inventory of period t
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]+self.Growth['Growth']
  

  def Williness_Harvest(self, Factor=0):
    l = len(self.Residues[0])
    c = int(l*Factor)
    Num_Williness = random.sample(range(l), c)
    retun(Num_Willines)
    

  def Initialize_Totals(self,Y):
    self.__class__.Total_Harvested = []
    self.__class__.Total_Residues = []
    for i in range(Y+1):
      self.__class__.Total_Harvested.append(pd.DataFrame({'Unique_ID' : [],'Harvested':[]})) #initialize Total Harvested variable (for all the layers)
      self.__class__.Total_Residues.append(pd.DataFrame({'Unique_ID' : [],'Harvested':[]})) #Initialize Total Residues variable (for all the layers)
  

  def Total_Harvested_Update(self,Y):
    for i in range(Y):
      self.Harvested[Y].loc[:,('Harvested')] = self.Harvested[Y].loc[:,('Harvested')] + self.Harvested[i].loc[:,('Harvested')]

    for i in range(Y+1):
      c = pd.concat([self.__class__.Total_Harvested[i], self.Harvested[i]]).groupby(['Unique_ID']).sum().reset_index() #updating Total Harvested
      self.__class__.Total_Harvested[i] = c.copy()


  def Total_Residues_Update(self,Y):
    for i in range(Y):
      self.Residues[Y].loc[:,('Harvested')] = self.Residues[Y].loc[:,('Harvested')] + self.Residues[i].loc[:,('Harvested')]

    for i in range(Y+1):
      c = pd.concat([self.__class__.Total_Residues[i], self.Residues[i]]).groupby(['Unique_ID']).sum().reset_index() #updating Total Harvested
      self.__class__.Total_Residues[i] = c.copy()


  def prueba(self):
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Distance = [0,1,2],[3,5,8]
    print('Type Distance', type(Distance))
    Demand = [100,200,300]
    Available =[400,500]
    Di = np.array([Distance])    
    #S = Distance.shape
    #print('S',S)
    De = np.array(Demand)    
    A = np.array(Available)    
    H = m.Array(m.Var,(2,3),value = 0,lb=0)
    
    for i in range(len(A)):
      m.Equation(np.sum(H[i,:])<=A[i])
    
    for j in range(len(De)):
      m.Equation(np.sum(H[:,j],axis=0)==De[j])
    m.Obj(-np.sum(H*Di))
    m.solve()
    print(H)
    
        
    

**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [34]:


# Main Program
YEARS = 3

start_time = time.time()
#Upload distances from each hexagon to other
Hardwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Saw_V3.csv")
Hardwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Pulp_V3.csv")
Softwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Saw_V3.csv")
Softwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Pulp_V3.csv")
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dist_Mill.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN_V2.csv")
Growth_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Growth_Hexagon.csv")
Aux = Growth_Data[(Growth_Data['OWNERSHIP']=='PRIVATE')&(Growth_Data['SLOPE']=='LE40')] #FILTER OWNERSHIP AND SLOPE
Growth_Data = Aux
Growth_Data = Growth_Data.fillna(0)

#Applying factors to have all the data in cubic feet
i1 = Mills_Data['plant'] == "Composite Panel"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Pulpmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Sawmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*83.3  #transforming mbf to cubic feet
i1 = Mills_Data['plant'] == "Venner"
Mills_Data.loc[i1,('Demand')] = 133280 #transforming mbf to cubic feet

Mills_Data[Mills_Data.Unique_ID==86399] #removing this mills with no data
Mills_Data.drop(index=106, inplace=True)
Mills_Data[Mills_Data.Unique_ID==86639] #removing this mills with no data
Mills_Data.drop(index=3, inplace=True)

Mixed = Mills_Data[Mills_Data['s_h']=='S/H'].copy()
Mixed.reset_index(inplace=True)
  
for i in range(len(Mixed)):
  print(Mixed.loc[i,'Unique_ID'])

#Creating Inventories Objects and Initializing
Hardwood_Saw = Inventory(Hardwood_Saw_Data,Dist_Data,Wt='H',Pt ='Sawmill')
Hardwood_Pulp = Inventory(Hardwood_Pulp_Data,Dist_Data,Wt='H',Pt ='Pulpmill')
Softwood_Saw = Inventory(Softwood_Saw_Data,Dist_Data,Wt='S',Pt ='Sawmill')
Softwood_Pulp = Inventory(Softwood_Saw_Data,Dist_Data,Wt='S',Pt ='Pulpmill')
Hardwood_Saw.Initialize_Totals(YEARS)

Hardwood_Saw.prueba()

#updating Mills According the wood type and product type
Hardwood_Saw.Mills_Update(Mills_Data)
Hardwood_Pulp.Mills_Update(Mills_Data)
Softwood_Saw.Mills_Update(Mills_Data)
Softwood_Pulp.Mills_Update(Mills_Data)


#Harvest Inventory Objects
#print('Total Harvested 0:',np.sum(Hardwood_Saw.__class__.Total_Harvested.Harvested))
Hardwood_Saw.Harvest_All_Mills_All_Years(YEARS)
Hardwood_Pulp.Harvest_All_Mills_All_Years(YEARS)
Softwood_Saw.Harvest_All_Mills_All_Years(YEARS)
Softwood_Pulp.Harvest_All_Mills_All_Years(YEARS)

start_time = time.time()



Hardwood_Saw.Total_Harvested_Update(YEARS)
Hardwood_Pulp.Total_Harvested_Update(YEARS)
Softwood_Saw.Total_Harvested_Update(YEARS)
Softwood_Pulp.Total_Harvested_Update(YEARS)

Hardwood_Saw.Total_Residues_Update(YEARS)
Hardwood_Pulp.Total_Residues_Update(YEARS)
Softwood_Saw.Total_Residues_Update(YEARS)
Softwood_Pulp.Total_Residues_Update(YEARS)


for y in range(YEARS):
  Hardwood_Saw.Harvested[y].to_csv('/content/drive/My Drive/Colab Notebooks/Results/year'+str(y)+'.csv')
  Hardwood_Saw.By_Mill.to_csv('/content/drive/My Drive/Colab Notebooks/Results/By_Mill'+str(y)+'.csv')


#Totalize the Wood Harvested 
#Cs =len(Hardwood_Saw.Harvested[YEARS])


for y in range(YEARS+1):
  print('TOTAL HARVESTED at Hardwood_saw at',y,'  ', np.sum(Hardwood_Saw.Harvested[y].loc[:,('Harvested')]))

for y in range(YEARS+1):
  print('TOTAL HARVESTED at Hardwood_Pulp at',y,'  ', np.sum(Hardwood_Pulp.Harvested[y].loc[:,('Harvested')]))



for y in range(YEARS+1):
  print('TOTAL HARVESTED at',y,'  ', np.sum(Hardwood_Saw.__class__.Total_Harvested[y].loc[:,('Harvested')]))

#print('TOTAL HARVESTED at',YEARS,'  ', np.sum(Hardwood_Saw.__class__.Total_Harvested[YEARS].loc[:,('Harvested')]))


#for y in range(YEARS):
#  print('Total Harvested at',y,'  ',Hardwood_Saw.Harvested[y].loc[:,('Harvested')].sum())



Hardwood_Saw.Harvested[YEARS].to_csv('/content/drive/My Drive/Colab Notebooks/Results/year'+str(YEARS)+'.csv')

#print ("*********1ST STAGE TOOK", time.time() - start_time, "TO RUN")

75595
88300
90199
101304
101041
Type Distance <class 'tuple'>
apm 34.82.131.34_gk_model94 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            8
   Intermediates:            0
   Connections  :            0
   Equations    :            6
   Residuals    :            6
 
 Number of state variables:              8
 Number of total equations: -            5
 Number of slack variables: -            2
 ---------------------------------------
 Degrees of freedom       :              1
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

*********************************

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


**Demanda: 8330.0 Disponible 72547022 Year:  0 Mill:  65748
Stage: 1
Cosechando Sawmill: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


**Demanda: 8330.0 Disponible 72538692 Year:  1 Mill:  65748
Stage: 2
Cosechando Sawmill: 0


...

59655 no hallado
60681 no hallado
Stage: 1
Cosechando Sawmill: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


**Demanda: 104960000.0 Disponible 0 Year:  1 Mill:  59860
59655 no hallado
60681 no hallado
Stage: 2
Cosechando Sawmill: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


**Demanda: 104960000.0 Disponible 0 Year:  2 Mill:  59860
59655 no hallado
60681 no hallado
***y: 2
TOTAL HARVESTED at Hardwood_saw at 0    8330.000000203125
TOTAL HARVESTED at Hardwood_saw at 1    8330.000000203221
TOTAL HARVESTED at Hardwood_saw at 2    8330.000000203318
TOTAL HARVESTED at Hardwood_saw at 3    24990.00000060966
TOTAL HARVESTED at Hardwood_Pulp at 0    8703999.95
TOTAL HARVESTED at Hardwood_Pulp at 1    8703999.93470005
TOTAL HARVESTED at Hardwood_Pulp at 2    8703999.9288
TOTAL HARVESTED at Hardwood_Pulp at 3    26111999.813500054
TOTAL HARVESTED at 0    98873271.95000018
TOTAL HARVESTED at 1    8720659.934700223
TOTAL HARVESTED at 2    8720659.928800173
TOTAL HARVESTED at 3    116314591.81350057


**SECOND STAGE**

In [30]:
start_time = time.time()
#Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
#Hardwood_Saw.Harvested_Res.append(pd.DataFrame(Data).copy())
#Hardwood_Saw.Optimal_Location_Total(YEARS,2)

print ("*********2ND STAGE TOOK", time.time() - start_time, "TO RUN")

*********2ND STAGE TOOK 2.5510787963867188e-05 TO RUN
